# T5를 활용해서 Generation-based MRC 문제를 풀어보기

두번째 미션에서는 **3강에서 배운 Generation based MRC**를 구현합니다!
> Remind !🤔  
* Extraction-based MRC : 지문 내 답의 위치를 예측 (classification) , 주어진 지문 내에 무조건 답이 존재함
* Generation-based MRC : 주어진 지문과 질의를 통해 답변을 생성함(Generation) , 주어진 지문 내에 답이 존재하지 않아도 되며 비교적 유연함

* 주어진 지문과 질의을 기반으로 적절한 답을 생성하는것이 오늘의 목표
* T5 모델을 활용하여 문제를 풀어볼 예정이니, T5에 대한 사전 지식이 필요합니다 
* T5는 encoder-decoder 구조의 모델이니 encoder만으로 구성된 BERT와는 구조가 상이합니다
* 해당 미션은 빈칸이 없습니다. 코드 한줄씩 살펴보면서 의미를 확인해보세요 !


```
🛠 Setup을 하는 부분입니다. 이전 과제에서 반복되는 부분이기 때문에 무지성 실행 하셔도 좋습니다.
💻 실습 코드입니다. 따라가면서 코드를 이해해보세요.
```

## 🛠 초기설정

### 🛠 Requirements

In [1]:
# Output model file을 저장하기위해 drive 마운트를 수행합니다
# 마운트를 진행하지 않은 경우, 모델의 크기가 커질 경우엔 디스크 용량이 부족하여 에러가 발생할 수 있습니다
try:
    from google.colab import drive
    drive.mount("/content/drive")
except ImportError:
    print('Runnning on local machine, not colab.')

Mounted at /content/drive


In [2]:
# Requirements
!pip install tqdm==4.64.1 -q
!pip install datasets==2.7.0 -q
!pip install transformers==4.24.0 -q
!pip install sentencepiece==0.1.97 -q
!pip install nltk -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import nltk
# nltk는 postprocess를 위해 import합니다 - postprocess_text 함수 참고
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 🛠 난수 고정 및 버전 확인

In [4]:
import random
import numpy as np
import torch

# pytorch version, device 정보 확인하기 
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ("device:[%s]."%(device))

PyTorch version:[2.0.0+cu118].
device:[cuda:0].


In [5]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    
set_seed(42) # magic number :)

### 🛠 데이터 및 평가 지표 불러오기

해당 데이터셋에 대해 미리 보고 싶으신분들은 해당 링크에서 확인해주세요:)  
[출처] https://huggingface.co/datasets/squad_kor_v1

In [6]:
from datasets import load_dataset

datasets = load_dataset("squad_kor_v1")

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from datasets import load_metric

metric = load_metric("squad")

<ipython-input-7-0eb82bc40e87>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")


##  💻 T5를 활용하여 Generation based QA 모델 구현하기

### 💻 Pre-trained 모델 및 토크나이저 불러오기

In [8]:
"""
[간단 설명]

1. huggingface의 transformers 라이브러리를 통해 모델을 불러옵니다
2. AutoModel을 사용하면 원하는 모델을 이름만 바꾸어 불러 올 수 있습니다
3. QuestionAnswering을 위한 모델이 이미 구축되어있기 때문에 편하게 불러와서 사용 할 수 있습니다
4. 대용량 데이터로 기학습된 모델을 불러와서 사용합니다 (기학습된 == Pretrained)
"""
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)

In [9]:
model_name = "paust/pko-t5-small"

In [10]:
# T5는 seq2seq 모델이므로 model을 불러올 때 AutoModelForSeq2SeqLM을 사용해야 함
config = AutoConfig.from_pretrained(
    model_name,
    cache_dir=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    config=config,
    cache_dir=None,
)

In [11]:
# model 구조 살펴보기
model 

T5ForConditionalGeneration(
  (shared): Embedding(50358, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

### 💻 파라미터 설정하기

In [12]:
# 데이터 전처리를 위한 파라미터 
max_source_length = 384
max_target_length = 128 # 최대 넘어가지 않도록 막아줌
padding = "max_length"

# 학습을 위한 파라미터
preprocessing_num_workers=12
num_beams = 2
max_train_samples = 5000
max_val_samples = 500
train_batch_size = 24
eval_batch_size = 2
num_train_epochs = 3

### 💻 전처리하기

In [13]:
def preprocess_function(examples):
    inputs = [f"question: {q}  context: {c} </s>" for q, c in zip(examples["question"], examples["context"])]
    targets = [f'{a["text"][0]} </s>' for a in examples['answers']]
    model_inputs = tokenizer(
        inputs,
        max_length=max_source_length,
        padding=padding,
        truncation=True
    )

    # targets(label)을 위해 tokenizer 설정
    labels = tokenizer(
        text_target=targets,
        max_length=max_target_length,
        padding=padding,
        truncation=True,
    )

    model_inputs["labels"] = labels["input_ids"] 
    model_inputs["example_id"] = []
    for i in range(len(model_inputs["labels"])):
        model_inputs["example_id"].append(examples["id"][i])
    return model_inputs

In [14]:
column_names = datasets["train"].column_names

In [15]:
# 전체 train dataset을 사용하는 예제가 아니고, sampling된 데이터를 사용하는 코드입니다. 적절하게 코드를 수정하여 사용하셔도 좋습니다.

train_dataset = datasets["train"]
train_dataset = train_dataset.select(range(max_train_samples))
train_dataset = train_dataset.map( #
    preprocess_function,
    batched=True,
    num_proc=preprocessing_num_workers,
    remove_columns=column_names,
    load_from_cache_file=False,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

#10:   0%|          | 0/1 [00:00<?, ?ba/s]

#11:   0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
# 전체 데이터로 평가
eval_examples = datasets["validation"]

# 샘플 데이터로 평가
eval_examples = eval_examples.select(range(max_val_samples)) 

eval_dataset = eval_examples.map(
    preprocess_function,
    batched=True,
    num_proc=preprocessing_num_workers,
    remove_columns=column_names,
    load_from_cache_file=False,
)


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

#10:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

#11:   0%|          | 0/1 [00:00<?, ?ba/s]

### 💻 Fine-tuning하기

In [17]:
from transformers import (
    DataCollatorForSeq2Seq, # 다른 시퀀스 length을 가진 input들을 합쳐줘서 gpu에서 pair computing이 쉽게 만들어 준다.
    Seq2SeqTrainer, # 
    Seq2SeqTrainingArguments #
)

In [18]:
label_pad_token_id = tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=None,
)

In [19]:
def postprocess_text(preds, labels):
  """
  postprocess는 nltk를 이용합니다.
  Huggingface의 TemplateProcessing을 사용하여
  정규표현식 기반으로 postprocess를 진행할 수 있지만
  해당 미션에서는 nltk를 이용하여 간단한 후처리를 진행합니다
  """

  preds = [pred.strip() for pred in preds]
  labels = [label.strip() for label in labels]
    
  preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
  labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

  return preds, labels

In [20]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds
  if isinstance(preds, tuple):
      preds = preds[0]
  
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  # decoded_labels은 rouge metric을 위한 것이며, f1/em을 구할 때 사용되지 않음
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # 간단한 post-processing
  decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

  formatted_predictions = [{"id": ex["id"], "prediction_text": decoded_preds[i]} for i, ex in enumerate(datasets["validation"].select(range(max_val_samples)))]
  references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"].select(range(max_val_samples))]

  result = metric.compute(predictions=formatted_predictions, references=references)
  return result

In [21]:
args = Seq2SeqTrainingArguments(
    output_dir='outputs', 
    do_train=True, 
    do_eval=True, 
    predict_with_generate=True,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    num_train_epochs=num_train_epochs,
    save_strategy='epoch',
    save_total_limit=2 # 모델 checkpoint를 최대 몇개 저장할지 설정
)

In [22]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [23]:
train_result = trainer.train(resume_from_checkpoint=None)

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: example_id. If example_id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 627
  Number of trainable parameters = 95628672
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a me

Step,Training Loss
500,0.269900


Saving model checkpoint to outputs/checkpoint-209
Configuration saved in outputs/checkpoint-209/config.json
Model weights saved in outputs/checkpoint-209/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-209/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-209/special_tokens_map.json
Saving model checkpoint to outputs/checkpoint-418
Configuration saved in outputs/checkpoint-418/config.json
Model weights saved in outputs/checkpoint-418/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-418/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-418/special_tokens_map.json
Saving model checkpoint to outputs/checkpoint-627
Configuration saved in outputs/checkpoint-627/config.json
Model weights saved in outputs/checkpoint-627/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-627/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-627/special_tokens_map.json
Deleting older checkpoint [o

In [24]:
train_result

TrainOutput(global_step=627, training_loss=0.2230048012315181, metrics={'train_runtime': 589.8099, 'train_samples_per_second': 25.432, 'train_steps_per_second': 1.063, 'total_flos': 2413842923520000.0, 'train_loss': 0.2230048012315181, 'epoch': 3.0})

### 💻 평가하기

In [25]:
metrics = trainer.evaluate(
    max_length=max_target_length,
    num_beams=num_beams,
    metric_key_prefix="eval"
)

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: example_id. If example_id are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 2


In [26]:
metrics

{'eval_loss': 0.03034132346510887,
 'eval_exact_match': 51.2,
 'eval_f1': 58.693809523809534,
 'eval_runtime': 47.3244,
 'eval_samples_per_second': 10.565,
 'eval_steps_per_second': 5.283,
 'epoch': 3.0}

In [27]:
def generarate_answer(sample):
    inputs = f'question: {sample["question"]}  context: {sample["context"]} </s>'
    print(inputs)
    sample = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True, return_tensors='pt')
    sample = sample.to("cuda:0")
    outputs = model.generate(**sample, max_length=max_target_length, num_beams=num_beams)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

    pred = "\n".join(nltk.sent_tokenize(pred))

    return pred

import numpy as np
np.random.seed(seed=7777) 

for i in np.random.randint(0, len(datasets["validation"]), 5):
    print(generarate_answer(datasets["validation"][int(i)]))
    print("=" * 8)

question: 유아인이 배우로서 처음으로 부산국제영화제에 참석한 년도는?  context: 2006년 1월 스크린 데뷔작인 독립영화 《우리에게 내일은 없다》의 촬영을 시작했다. 이 영화를 연출한 노동석 감독은 오디션을 볼 당시 유아인에게 극 중 캐릭터에 대해 묻자 창 밖을 한참 바라보며 “슬프죠”라는 한 마디만을 던진 모습이 인상적이었다며 캐스팅의 이유를 밝혔다. 유아인은 이 영화에서 진짜 총을 구해 현실로부터 자신을 구해내려는 소년 ‘종대’ 역할을 맡았는데, 인터뷰에서 "종대처럼 사건에 휘말린 적도 없고 불우한 환경에서 자라지도 않았지만 제가 종대와 비슷한 시기에 느꼈던 불안이나 두려움 등이 연기를 하는 데 큰 도움이 됐습니다. 종대도 청춘이고 저도 청춘이니까요"라며 연기를 한 소회를 밝혔다. 2007년 5월 《우리에게 내일은 없다》 언론시사회에서는 작품에 대해 “배우라는 앞날에 대한 꿈을 꾸고 그림을 그렸다면 그 그림 속에 꼭 있어야 할 영화”라며 본인의 영화 데뷔작에 대한 애정을 드러낸다. 또한 배우로서 고유한 소년성을 갖게해 준 ‘첫 활시위’ 같은 작품이라고 설명한다. 2006년 10월 유아인은 이 영화를 통해 배우로서 처음으로 부산국제영화제 개막식과 GV에 참석한다. </s>
2007년
question: 김희선이 4년만에 브라운관에 컴백한 인기 드라마 <야마토 나데시코>를 원작으로 한 로맨스 드라마는?  context: 김희선은 2000년대에 접어들면서 스크린으로 활동 무대를 옮겨 드라마 출연을 한동안 중단하였다. 영화 《와니와 준하》(2001), 《화성으로 간 사나이》(2003)에 출연했지만 번번히 이렇다 할 흥행을 거두지 못한 채 2003년 일본의 인기 드라마 《야마토 나데시코》를 원작으로 한 로맨스 드라마 《요조숙녀》로 4년여만에 브라운관에 컴백하였다. 하지만 이 작품은 진부한 설정과 스토리로 기대 이상의 주목은 받지 못했다. 이듬해, 2004년에는 한류를 겨냥한 멜로 드라마 《슬픈 연가》에서 출연하였지만 남자주인공 중 한 명인 송승헌이 병역

### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

